In [1]:
from google.cloud import bigquery
from google.cloud import storage
from google import genai
from google.colab import auth

### Initialize Clients

Sets up the required clients to interact with Google Cloud services

In [2]:
PROJECT_ID = "qwiklabs-gcp-03-be6b72ae4b0d"

#Big Query client
bq_client = bigquery.Client(project=PROJECT_ID)

# Create Gemini client using Vertex AI
gemini_client = genai.Client(
    vertexai=True,
    project="qwiklabs-gcp-03-be6b72ae4b0d",
    location="global"
)

print("Bigquery and Gemini clients Initialized successfully")

Bigquery and Gemini clients Initialized successfully


### Load Aurora Bay FAQ Data

Create a BigQuery dataset (`aurora_bay_rag`) and loads a CSV file containing FAQs from a public GCS bucket into a table named `faq_data`.

In [3]:
# Load Aurora Bay FAQs from GCS to BigQuery

from google.cloud import bigquery

# Define dataset and table
dataset_id = "aurora_bay_rag"
table_id = "faq_data"
table_ref = f"{PROJECT_ID}.{dataset_id}.{table_id}"

# Create the dataset (if it doesn't exist)
dataset_ref = bigquery.Dataset(f"{PROJECT_ID}.{dataset_id}")
dataset_ref.location = "US"
bq_client.create_dataset(dataset_ref, exists_ok=True)

# CSV source in GCS
gcs_uri = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"

# Load configuration
job_config = bigquery.LoadJobConfig(
    autodetect=True,
    skip_leading_rows=1,
    source_format=bigquery.SourceFormat.CSV
)

# Load job
load_job = bq_client.load_table_from_uri(
    gcs_uri,
    table_ref,
    job_config=job_config
)

load_job.result()

print(f"Data loaded successfully into: {table_ref}")


Data loaded successfully into: qwiklabs-gcp-03-be6b72ae4b0d.aurora_bay_rag.faq_data


### Create remote embedding model using BigQuery ML
Note: Before running the query, make sure to,
1. Create External Connection named embedding_conn in Big Query Studio
2. Grant the service account the necessary IAM permissions (BigQuery Data Owner and Vertex AI User)

In [4]:
embedding_model_query = """
CREATE OR REPLACE MODEL `aurora_bay_rag.faq_embedding_model`
REMOTE WITH CONNECTION `us.embedding_conn`
OPTIONS (ENDPOINT = 'text-embedding-005');
"""

job = bq_client.query(embedding_model_query)
job.result()

print("Embedding model created: aurora_bay_rag.faq_embedding_model")

Embedding model created: aurora_bay_rag.faq_embedding_model


### (Optional) Preview the column and first few rows of the table

In [5]:
table = bq_client.get_table("aurora_bay_rag.faq_data")

preview_df = bq_client.query("""
SELECT * FROM `aurora_bay_rag.faq_embedded`
LIMIT 5
""").to_dataframe()

# preview_df.head()

### Generate embeddings and store them in a new table

Generate the embedding from the `faq_data` table to new `faq_embedded` table.

In [6]:
generate_embed_sql = """
CREATE OR REPLACE TABLE `aurora_bay_rag.faq_embedded` AS
SELECT *,
       ml_generate_embedding_result AS embedding
FROM ML.GENERATE_EMBEDDING(
  MODEL `aurora_bay_rag.faq_embedding_model`,
  (
    SELECT
      CONCAT(string_field_0, ' ', string_field_1) AS content,
      string_field_0 AS question,
      string_field_1 AS answer
    FROM `aurora_bay_rag.faq_data`
  )
);
"""

bq_client.query(generate_embed_sql).result()

print("Embeddings generated and stored in: aurora_bay_rag.faq_embedded")


Embeddings generated and stored in: aurora_bay_rag.faq_embedded


### (Optional) Preview contents of the Embeddings table



In [7]:
# Run only if you need to view the columns in embedding table
table = bq_client.get_table("aurora_bay_rag.faq_embedded")
print("Columns in aurora_bay_rag.faq_embedded:")
for field in table.schema:
    print(f"- {field.name} ({field.field_type})")

Columns in aurora_bay_rag.faq_embedded:
- ml_generate_embedding_result (FLOAT)
- ml_generate_embedding_statistics (JSON)
- ml_generate_embedding_status (STRING)
- content (STRING)
- question (STRING)
- answer (STRING)
- embedding (FLOAT)


### Perform a Vector Search for the user query.

In [8]:
def search_faq_results(user_question: str, top_k: int = 3):
    query = f"""
    SELECT
      query.query,
      result.base.question,
      result.base.answer,
      result.distance
    FROM VECTOR_SEARCH(
      TABLE `aurora_bay_rag.faq_embedded`,
      'embedding',
      (
        SELECT
          ml_generate_embedding_result AS embedding,
          '{user_question}' AS query
        FROM ML.GENERATE_EMBEDDING(
          MODEL `aurora_bay_rag.faq_embedding_model`,
          (SELECT '{user_question}' AS content)
        )
      ),
      top_k => {top_k},
      options => '{{"fraction_lists_to_search": 1.0}}'
    ) AS result
    """
    return bq_client.query(query).to_dataframe()

### Generate Answer from gemini with the FAQ context

This function takes the user’s question and a list of similar FAQs (from the vector search),
and constructs a prompt for the Gemini.

Gemini uses the FAQ context to generate a friendly and helpful response.
It ensures the assistant stays relevant to Aurora Bay and gives natural-language answers.

In [9]:
def generate_answer_from_gemini(user_question: str, faq_context_df):
    # Format the FAQ context for Gemini
    context_text = "\n".join([
        f"Q: {row['question']}\nA: {row['answer']}" for _, row in faq_context_df.iterrows()
    ])

    prompt = f"""
You are a helpful assistant for the town of Aurora Bay, Alaska.
Use the frequently asked questions below to answer the user question. Do not answer questions that are not in the context.
Say I am unable to answer the question if you do not know the answer.

FAQs:
{context_text}

User question: {user_question}

Answer in a clear, concise, and friendly tone:
"""

    contents = [
        genai.types.Content(
            role="user",
            parts=[genai.types.Part(text=prompt)]
        )
    ]

    response = gemini_client.models.generate_content(
        model="gemini-2.5-pro-preview-06-05",
        contents=contents,
        config=genai.types.GenerateContentConfig(
            temperature=0.7,
            max_output_tokens=1024
        )
    )

    if response.candidates and response.candidates[0].content.parts:
        return response.candidates[0].content.parts[0].text.strip()
    else:
        return "Oops! Error fetching response"


### Chatbot Interface (Optional to use)

Just call the `generate_answer_from_gemini("Where can I pay my water bill?")` function with your query to get the answers from RAG.

Optionally use the below interface to get a conversational chatbot like experience.

This function starts a text-based chatbot loop. The assistant:
- Greets the user
- Accepts their question
- Uses vector search + Gemini to generate a response
- Asks if the user wants to see FAQ sources or ask another question
- Lets the user exit any time by typing 'exit'

In [ ]:
generate_answer_from_gemini("Where can I pay my water bill?")

In [10]:
def start_aurora_bay_chat():
    print("Hi! I’m an assistant for Aurora Bay, Alaska.\n\nWhat would you like to know? (Type 'exit / end' to end)\n")

    while True:
        user_question = input("You: ").strip()
        if user_question.lower() in ["exit", "quit"]:
            print("Thanks for chatting! Have a great day in Aurora Bay")
            break

        # Step 1: Vector Search
        faq_matches = search_faq_results(user_question)

        # Step 2: Gemini Response
        answer = generate_answer_from_gemini(user_question, faq_matches)
        print("\n Gemini: " + answer + "\n")

        # Step 3: Ask for resource view or next
        next_action = input("Would you like to view the sources? Type 'yes' to see FAQs, 'next' to ask another question, or 'exit / end' to quit: ").strip().lower()

        if next_action == "yes":
            print("\nTop Matching FAQs:")
            for idx, row in faq_matches.iterrows():
                print(f"\nQ{idx+1}: {row['question']}\nA{idx+1}: {row['answer']}\n")

        elif next_action == "exit" or next_action == 'end' :
            print("\nGoodbye! Feel free to come back with more questions.")
            break

        print("\nReady for your next question!\n")

start_aurora_bay_chat()


Hi! I’m an assistant for Aurora Bay, Alaska.

What would you like to know? (Type 'exit / end' to end)

You: Where can I pay my water bill?

 Gemini: Hello! You can pay your water bill online, by mail, or in person at the Aurora Bay Utilities Department located within Town Hall.

Would you like to view the sources? Type 'yes' to see FAQs, 'next' to ask another question, or 'exit / end' to quit: yes

Top Matching FAQs:

Q1: Where can I pay my water and sewer bills?
A1: You can pay water and sewer bills online, by mail, or in person at the Aurora Bay Utilities Department located within Town Hall.


Q2: Where can I pay my water and sewer bills?
A2: You can pay water and sewer bills online, by mail, or in person at the Aurora Bay Utilities Department located within Town Hall.


Q3: Where can I pay my water and sewer bills?
A3: You can pay water and sewer bills online, by mail, or in person at the Aurora Bay Utilities Department located within Town Hall.


Ready for your next question!

You: